Данное демо включает в себя демонстрацию работы вокодера. Тут присутствует импорт нужных библиотек, показано где и откуда скачивать веса модели, а также как запускать саму модель.

Клонирование репозитория и установка зависимостей

In [ ]:
!git clone https://github.com/aksenovmr/hifigan-vocoder-hw3.git
%cd hifigan-vocoder-hw3

!pip install -r requirements.txt
!pip install huggingface_hub soundfile

fatal: destination path 'hifigan-vocoder-hw3' already exists and is not an empty directory.
/content/hifigan-vocoder-hw3


Скачивание чекпоинта модели с huggingface

In [ ]:
from huggingface_hub import hf_hub_download

CHECKPOINT_PATH = hf_hub_download(
    repo_id="aksenovmr/hifigan-vocoder-hw3",
    filename="checkpoint-epoch120.pth")

print("Checkpoint downloaded:", CHECKPOINT_PATH)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


checkpoint-epoch120.pth:   0%|          | 0.00/672M [00:00<?, ?B/s]

Checkpoint downloaded: /root/.cache/huggingface/hub/models--aksenovmr--hifigan-vocoder-hw3/snapshots/50e334569faedcff6846218fa3d295082f8c28d1/checkpoint-epoch120.pth


Загрузка модели вокодера

In [ ]:
import torch
from src.model.hifigan.generator import HiFiGANGeneratorV2

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

generator = HiFiGANGeneratorV2().to(device)

checkpoint = torch.load(CHECKPOINT_PATH, map_location=device)

generator.load_state_dict(checkpoint["generator"])

generator.eval()

print("Модель загружена")

/usr/local/lib/python3.12/dist-packages/torch/nn/utils/weight_norm.py:134: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
/tmp/ipython-input-2823330176.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case w

Модель загружена


Получение mel-спектрограмм

In [ ]:
from src.transforms.mel_spectrogram import (
    MelSpectrogramTransform,
    MelSpectrogramConfig)

mel_config = MelSpectrogramConfig()
mel_transform = MelSpectrogramTransform(mel_config).to(device)

Функция для синтеза аудио из мел спектрограмм

In [ ]:
import torchaudio
from IPython.display import Audio, display
from pathlib import Path

@torch.no_grad()
def synthesize(wav_path):
    waveform, sr = torchaudio.load(wav_path)

    if sr != mel_config.sample_rate:
        waveform = torchaudio.functional.resample(
            waveform, sr, mel_config.sample_rate
        )

    waveform = waveform.to(device)

    mel = mel_transform(waveform)
    generated = generator(mel)

    return (waveform.squeeze().detach().cpu(), generated.squeeze().detach().cpu())

Загрузка исходных аудиофайлов для тестирования

In [ ]:
!mkdir mos_wavs
!wget -P mos_wavs https://raw.githubusercontent.com/aksenovmr/hifigan-vocoder-hw3/main/demo/mos_ground_truth/1.wav
!wget -P mos_wavs https://raw.githubusercontent.com/aksenovmr/hifigan-vocoder-hw3/main/demo/mos_ground_truth/2.wav
!wget -P mos_wavs https://raw.githubusercontent.com/aksenovmr/hifigan-vocoder-hw3/main/demo/mos_ground_truth/3.wav

--2026-02-19 09:55:04--  https://raw.githubusercontent.com/aksenovmr/hifigan-vocoder-hw3/main/demo/mos_ground_truth/1.wav
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 296684 (290K) [audio/wav]
Saving to: ‘mos_wavs/1.wav’

1.wav               100%[===================>] 289.73K  --.-KB/s    in 0.03s   

2026-02-19 09:55:04 (8.72 MB/s) - ‘mos_wavs/1.wav’ saved [296684/296684]

--2026-02-19 09:55:05--  https://raw.githubusercontent.com/aksenovmr/hifigan-vocoder-hw3/main/demo/mos_ground_truth/2.wav
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 2

Синтез аудио и получение generated записей

In [ ]:
mos_dir = Path("mos_wavs")

for wav_file in sorted(mos_dir.glob("*.wav")):
    print("Файл:", wav_file.name)

    real, generated = synthesize(wav_file)

    print("Ground Truth:")
    display(Audio(real.numpy(), rate=mel_config.sample_rate))

    print("Generated:")
    display(Audio(generated.numpy(), rate=mel_config.sample_rate))

Файл: 1.wav
Ground Truth:


Generated:


Файл: 2.wav
Ground Truth:


Generated:


Файл: 3.wav
Ground Truth:


Generated:


В качестве альтернативы для запуска синтезатора можно использовать код ниже (качество получаемых аудио выходит выше). Он берет файлы из склонированного репозитория по пути demo/mos_ground_truth и выдает свой результат генерации по пути demo/demo_output, откуда можно скачать и прослушать аудио локально на компьютере.

In [ ]:
!python synthesize.py \
    --config src/configs/hifigan.yaml \
    --checkpoint $CHECKPOINT_PATH \
    --input_dir demo/mos_ground_truth \
    --output_dir demo/demo_output

/usr/local/lib/python3.12/dist-packages/torch/nn/utils/weight_norm.py:134: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
/content/hifigan-vocoder-hw3/synthesize.py:53: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any